In [8]:
from pdf2image import convert_from_path
import csv
import glob
import os

In [2]:
# File and Directory Paths
dataset_file = "extracted_tables/tables_regex.csv"
table_dir = "extracted_tables/table_code/"
header_dir = "extracted_tables/table_header/"
output_dir = "extracted_tables/table_images/"

if os.path.isdir(table_dir) == False:
    os.mkdir(table_dir)

if os.path.isdir(header_dir) == False:
    os.mkdir(header_dir)
    
if os.path.isdir(output_dir) == False:
    os.mkdir(output_dir)

In [12]:
# Directory for temporary latex files
if os.path.isdir(output_dir + "tmp/") == False:
    os.mkdir(output_dir + "tmp/")

with open(dataset_file, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    for row in spamreader:
        table_id = row[0]
        paper_id = row[1]
    
        # Getting the table code
        with open(table_dir + table_id + ".txt", "r") as file_table:
            table = file_table.read()
        
        # Getting the paper header code
        with open(header_dir + paper_id + ".txt", "r") as file_header:
            header = file_header.read()
        
        # Joining header and table
        tex_code = header + "\n\pagenumbering{gobble}\n" + table + "\n\end{document}"
        
        # Storing latex code in tex file
        code_path = output_dir + "tmp/table.tex"        
        with open(code_path, "w") as file_code:
            file_code.write(tex_code)
            
        # Using pdflatex to generate pdf file
        os.system(f"pdflatex -interaction=nonstopmode -output-directory={output_dir}tmp/ {code_path}")
        
        # Converting pdf file to an image and trimming whitespace
        images = convert_from_path(output_dir + "tmp/table.pdf")
        images[0].save(output_dir + "tmp/table.png", "PNG")    
        os.system(f"magick {output_dir}tmp/table.png -trim {output_dir + table_id}.png")
        
        # Deleting temporary latex files
        files = glob.glob(output_dir + "tmp/*")
        for f in files:
            os.remove(f)
        
if os.path.isdir(output_dir + "tmp/"):
    os.rmdir(output_dir + "tmp/")